## **Mission 2: Modélisation et tracking des expériences**


### Objectifs principaux :

Construire un modèle de régression linéaire comme baseline.

Explorer des modèles avancés (Random Forest, Gradient Boosting).

Suivre les expériences avec MLflow.

Enregistrer le meilleur modèle.

Dans cette section, nous nous concentrons sur l’évaluation des performances des différents modèles de régression en utilisant plusieurs configurations de paramètres. L’objectif principal est de comparer les modèles sur la base de métriques pertinentes telles que l’erreur quadratique moyenne (MSE), la racine carrée de l’erreur quadratique moyenne (RMSE), l’erreur absolue moyenne (MAE) et le coefficient de détermination (R²).

### **1- Régression linéaire**

Nous commencerons par construire un modèle de régression linéaire servant de baseline. Ce modèle, simple et interprétable, fournira une première référence pour évaluer la qualité des prédictions. Ensuite, nous explorerons des modèles plus avancés, notamment le Random Forest Regressor et le Gradient Boosting Regressor

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
import mlflow.sklearn
import numpy as np

# Charger les données
file_path = '../data/processed/housing_standardized.csv'
data = pd.read_csv(file_path)

# Définir (features) et la variable cible (target)
X = data.drop(columns=['MedHouseVal'])
y = data['MedHouseVal']

# Diviser les données en ensemble d'entraînement et de tests
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Début de l'expérience MLflow
mlflow.set_experiment("California Housing Linear Regression")

with mlflow.start_run(run_name="Linear Regression Model"):
    # Initialiser et entraîner le modèle
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Prédictions
    y_pred = model.predict(X_test)

    # Calcul des métriques
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)  # Racine carrée du MSE
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Enregistrer les paramètres
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("random_state", 42)

    # Enregistrer les métriques
    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("R-squared", r2)

    # Afficher les métriques
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"R-squared: {r2}")

    # Enregistrer le modèle
    mlflow.sklearn.log_model(model, "linear_regression_model")
    print("Modèle enregistré dans MLflow.")

2025/01/17 18:11:51 INFO mlflow.tracking.fluent: Experiment with name 'California Housing Linear Regression' does not exist. Creating a new experiment.


Mean Squared Error (MSE): 0.4059262952701108
Root Mean Squared Error (RMSE): 0.63712345371216
Mean Absolute Error (MAE): 0.4714992246236078
R-squared: 0.6522131659556444


2025/01/17 18:12:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modèle enregistré dans MLflow.


MSE (0.4059) et RMSE (0.6371) : Ces valeurs indiquent des erreurs relativement élevées dans les prédictions, montrant que le modèle n'est pas très précis.

MAE (0.4715) : L'écart moyen entre les prédictions et les valeurs réelles est de 0.4715, ce qui est notable.

R² (0.6522) : Le modèle explique environ 65% de la variance des données, ce qui est raisonnable, mais il y a encore de la place pour améliorer la précision.

### **2-Random Forest Model**

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
import mlflow.sklearn
import numpy as np

# Charger les données
file_path = '../data/processed/housing_data.csv'
data = pd.read_csv(file_path)

# Définir (features) et la variable cible (target)
X = data.drop(columns=['MedHouseVal'])
y = data['MedHouseVal']

# Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Début de l'expérience MLflow
mlflow.set_experiment("California Housing Random Forest")

with mlflow.start_run(run_name="Random Forest Model") as run:
    # Initialiser et entraîner le modèle Random Forest
    model = RandomForestRegressor(n_estimators=80, random_state=42)
    model.fit(X_train, y_train)

    # Prédictions
    y_pred = model.predict(X_test)

    # Calcul des métriques
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)  # Racine carrée du MSE
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Enregistrer les paramètres
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("n_estimators", 80)

    # Enregistrer les métriques
    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("R-squared", r2)

    # Afficher les métriques
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"R-squared: {r2}")

    # Enregistrer le modèle dans les artifacts
    model_name = "random_forest_model"
    mlflow.sklearn.log_model(model, model_name)
    print("Modèle enregistré dans les artifacts de MLflow.")

    # Enregistrer dans le Model Registry
    registered_model_name = "CaliforniaHousingRandomForest"
    model_uri = f"runs:/{run.info.run_id}/{model_name}"
    mlflow.register_model(model_uri=model_uri, name=registered_model_name)
    print(f"Modèle enregistré dans le Model Registry avec le nom : {registered_model_name}")

2025/01/17 18:21:11 INFO mlflow.tracking.fluent: Experiment with name 'California Housing Random Forest' does not exist. Creating a new experiment.


Mean Squared Error (MSE): 0.23157209638015866
Root Mean Squared Error (RMSE): 0.48121938487571203
Mean Absolute Error (MAE): 0.31549043595747844
R-squared: 0.8015951979669647


2025/01/17 18:21:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modèle enregistré dans les artifacts de MLflow.
Modèle enregistré dans le Model Registry avec le nom : CaliforniaHousingRandomForest


Successfully registered model 'CaliforniaHousingRandomForest'.
Created version '1' of model 'CaliforniaHousingRandomForest'.


Le MSE mesure l'erreur quadratique moyenne entre les valeurs prédites et les valeurs réelles. Une valeur de 0.2316 indique que l'erreur moyenne quadratique est faible.

Le RMSE (racine carrée du MSE) permet une interprétation plus intuitive puisqu'il est exprimé dans la même unité que la variable cible.
Une valeur de 0.4812 montre une erreur relativement faible dans les prédictions.

La MAE indique une erreur moyenne absolue de 0.3155, ce qui signifie que les prédictions s'écartent en moyenne de 0.3155 de la valeur réelle.

Le R² montre que le modèle explique environ 80% de la variance des données cibles, ce qui reflète une bonne capacité explicative.

### **3- Gradient Boosting**

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
import mlflow.sklearn
import numpy as np

# Charger les données
file_path = '../data/processed/housing_standardized.csv'
data = pd.read_csv(file_path)

# Définir (features) et la variable cible (target)
X = data.drop(columns=['MedHouseVal'])
y = data['MedHouseVal']

# Diviser les données en ensemble d'entraînement et de tests
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Début de l'expérience MLflow
mlflow.set_experiment("California Housing Gradient Boosting")

with mlflow.start_run(run_name="Gradient Boosting Model"):
    # Initialiser et entraîner le modèle Gradient Boosting
    model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Prédictions
    y_pred = model.predict(X_test)

    # Calcul des métriques
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)  # Racine carrée du MSE
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Enregistrer les paramètres
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("n_estimators", 100)

    # Enregistrer les métriques
    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("R-squared", r2)

    # Afficher les métriques
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"R-squared: {r2}")

    # Enregistrer le modèle
    mlflow.sklearn.log_model(model, "gradient_boosting_model")
    print("Modèle enregistré dans MLflow.")

2025/01/17 18:22:26 INFO mlflow.tracking.fluent: Experiment with name 'California Housing Gradient Boosting' does not exist. Creating a new experiment.


Mean Squared Error (MSE): 0.25665392313463087
Root Mean Squared Error (RMSE): 0.5066102280201524
Mean Absolute Error (MAE): 0.35231127289581704
R-squared: 0.7801057570989314


2025/01/17 18:22:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modèle enregistré dans MLflow.


Le MSE est légèrement plus élevé que celui de Random Forest, traduisant des prédictions un peu moins précises.

Le RMSE, à 0.5066, reflète une erreur modérée dans les prédictions.

La MAE est de 0.3523, une performance correcte mais inférieure à Random Forest.

Le R² indique que le modèle explique 78% de la variance des données, ce qui est une bonne performance mais légèrement inférieure à Random Forest.

###  Choix du meilleur modèle

Après avoir comparé les résultats des trois modèles, le Random Forest est le modèle qui offre les meilleures performances globales :

Erreurs plus faibles : Les métriques MSE, RMSE et MAE sont toutes inférieures, ce qui indique une précision accrue dans les prédictions.
R² supérieur : Random Forest explique 80% de la variance des données, surpassant Gradient Boosting (78%) et la Régression Linéaire (65%).
Robustesse : Random Forest est connu pour bien s'adapter aux ensembles de données complexes et variés.

En conclusion, bien que la régression linéaire soit simple et interprétable, et que le Gradient Boosting ait des performances correctes, le Random Forest est clairement le choix optimal pour cette tâche grâce à ses meilleures performances sur l'ensemble des métriques.